# Long text summarization using LCEL chains on Langchain with Bedrock APIs

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

## Overview
When we work with large documents, we can face some challenges as the input text might not fit into the model context length, or the model hallucinates with large documents, or, out of memory errors, etc.

To solve those problems, we are going to show a solution that is based on the concept of chunking and chaining prompts. This solution is leveraging [LangChain](https://python.langchain.com/docs/get_started/introduction.html) which is a popular framework for developing applications powered by language models.

In this architecture:

1. A large document (or a giant file appending small ones) is loaded
1. Langchain utility is used to split it into multiple smaller chunks (chunking)
1. First chunk is sent to the model; Model returns the corresponding summary
1. Langchain gets next chunk and appends it to the returned summary and sends the combined text as a new request to the model; the process repeats until all chunks are processed
1. In the end, you have final summary based on entire content

### Use case
This approach can be used to summarize call transcripts, meetings transcripts, books, articles, blog posts, and other relevant content.

### Imports

In [ ]:
import os
import sys
from langchain_aws import ChatBedrockConverse
from IPython.core.display import HTML
from IPython.display import display_markdown, Markdown
import boto3

HTML("<script>Jupyter.notebook.kernel.restart()</script>")

module_path = ".."
sys.path.append(os.path.abspath(module_path))

boto3_bedrock = boto3.client('bedrock-runtime')

textgen_llm = ChatBedrockConverse(
    model_id="us.amazon.nova-micro-v1:0",
    client=boto3_bedrock,
    max_tokens=None,
    temperature=0.5
)


### Load shareholder letter

We will be following a process similar to lab 02 in this summarization section. First, let us load the 2022 Amazon shareholder letter

In [ ]:
shareholder_letter = "./letters/2022-letter.txt"

with open(shareholder_letter, "r") as file:
    letter = file.read()

In [ ]:
len(letter.split(' '))

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n"], chunk_size=8096, chunk_overlap=100
)

docs = text_splitter.create_documents([letter])

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.output_parsers import XMLOutputParser
from langchain.schema.output_parser import StrOutputParser


xml_parser = XMLOutputParser(tags=['insight'])
str_parser = StrOutputParser()

prompt = PromptTemplate(
    template="""
    
    Human:
    {instructions} : \"{document}\"
    Format help: {format_instructions}.
    Assistant:""",
    input_variables=["instructions","document"],
    partial_variables={"format_instructions": xml_parser.get_format_instructions()},
)

insight_chain = prompt | textgen_llm | StrOutputParser()

In [ ]:
print(f"Number of Documents {len(docs)}")

# Option 1. Manually process insights, then summarize

In [ ]:
%%time
insights=[]
for i in range(len(docs)):
    insights.append(
        insight_chain.invoke({
        "instructions":"Provide Key insights from the following text",
        "document": {docs[i].page_content}
    }))

In [ ]:
str_parser = StrOutputParser()

prompt = PromptTemplate(
    template="""
    
    Human:
    {instructions} : \"{document}\"
    Assistant:""",
    input_variables=["instructions","document"]
)

summary_chain = prompt | textgen_llm | StrOutputParser()

In [ ]:
%%time
display_markdown(Markdown(summary_chain.invoke({
        "instructions":"You will be provided with multiple sets of insights. Compile and summarize these insights and provide key takeaways in one concise paragraph. Do not use the original xml tags. Just provide a paragraph with your compiled insights.",
        "document": {'\n'.join(insights)}
    })))

# Option 2. Use Map reduce pattern on Langchain

In [ ]:
from langchain.chains.summarize import load_summarize_chain
summary_chain = load_summarize_chain(llm=textgen_llm, chain_type="map_reduce", verbose=False, token_max=1024)

In [ ]:
%%time
display_markdown(Markdown(summary_chain.invoke(docs)['output_text']))

# Reference - Read Full Shareholder Letter
Optionally here please run the next cell to view the Shareholder letter. Cross reference with the outputs of options 1 and 2 to gage the effectiveness of the summerization prompts


In [ ]:
print(letter)